In [24]:
# Import necessary libraries
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import Tk, Label, Entry, Button, filedialog, messagebox, Frame
from tkinter import ttk
import shutil
import threading
import time
import csv

In [25]:
# Constants
DATASET_PATH = 'dataset'
TRAINER_PATH = 'trainer'
MODEL_FILE = os.path.join(TRAINER_PATH, 'trainer.yml')
ATTENDANCE_FILE = 'attendance.csv'
LOG_FILE = 'system_log.txt'

# Ensure necessary directories exist
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(TRAINER_PATH, exist_ok=True)

# Initialize Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
if face_cascade.empty():
    raise Exception("Failed to load Haar Cascade classifier")

In [26]:
# Helper functions for logging
def log_activity(message):
    """Log activity with timestamp to the log file"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f"[{timestamp}] {message}\n"
    
    try:
        with open(LOG_FILE, 'a') as f:
            f.write(log_message)
    except Exception as e:
        print(f"Failed to write to log file: {str(e)}")
    
    print(log_message.strip())

In [27]:
class CSVAttendanceSystem:
    def __init__(self, dataset_path=DATASET_PATH, model_file=MODEL_FILE, attendance_file=ATTENDANCE_FILE):
        self.dataset_path = dataset_path
        self.model_file = model_file
        self.attendance_file = attendance_file
        self.face_cascade = face_cascade
        try:
            self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        except AttributeError:
            raise Exception("OpenCV face recognition module not available. Ensure opencv-contrib-python is installed.")
        self.labels = {}
        self.confidence_threshold = 80  # Default threshold for face recognition
        self.status_callback = None
        self.attendance_thread = None
        self.stop_flag = False

    def set_status_callback(self, callback):
        """Set callback function for status updates"""
        self.status_callback = callback

    def update_status(self, message):
        """Update status through callback if available"""
        if self.status_callback:
            self.status_callback(message)
        log_activity(message)

    def collect_training_data(self):
        """Collect training data from the dataset directory"""
        faces = []
        ids = []
        label_id = 0
        self.labels = {}
        
        # Check if dataset path exists
        if not os.path.exists(self.dataset_path):
            self.update_status("[ERROR] Dataset path does not exist.")
            return faces, ids
        
        # Iterate through each person's directory
        for person_name in os.listdir(self.dataset_path):
            person_dir = os.path.join(self.dataset_path, person_name)
            if os.path.isdir(person_dir):
                self.labels[label_id] = person_name
                image_count = 0
                
                # Process each image in the person's directory
                for img_file in os.listdir(person_dir):
                    if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        continue
                        
                    img_path = os.path.join(person_dir, img_file)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    
                    if img is None:
                        self.update_status(f"[WARNING] Could not read image: {img_path}")
                        continue
                    
                    # Resize image for consistency
                    img = cv2.resize(img, (100, 100))
                    faces.append(img)
                    ids.append(label_id)
                    image_count += 1
                
                self.update_status(f"[INFO] Collected {image_count} images for {person_name}")
                label_id += 1
        
        # Save labels mapping to file for later use
        try:
            with open(os.path.join(TRAINER_PATH, 'labels.csv'), 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['ID', 'Name'])
                for label_id, name in self.labels.items():
                    writer.writerow([label_id, name])
        except Exception as e:
            self.update_status(f"[ERROR] Failed to save labels: {str(e)}")
        
        return faces, ids

    def train_model(self):
        """Train the face recognition model"""
        self.update_status("[INFO] Training model...")
        
        faces, ids = self.collect_training_data()
        
        if len(faces) == 0:
            self.update_status("[ERROR] No training data found.")
            return False
        
        try:
            self.recognizer.train(faces, np.array(ids))
            self.recognizer.save(self.model_file)
            self.update_status(f"[INFO] Model trained successfully with {len(faces)} images.")
            return True
        except Exception as e:
            self.update_status(f"[ERROR] Training failed: {str(e)}")
            return False

    def load_model(self):
        """Load the trained model and labels"""
        try:
            # Load the model file
            if os.path.exists(self.model_file):
                self.recognizer.read(self.model_file)
                
                # Load labels from file
                self.labels = {}
                labels_file = os.path.join(TRAINER_PATH, 'labels.csv')
                
                if os.path.exists(labels_file):
                    df = pd.read_csv(labels_file)
                    for _, row in df.iterrows():
                        self.labels[row['ID']] = row['Name']
                    
                    self.update_status("[INFO] Model and labels loaded successfully.")
                    return True
                else:
                    self.update_status("[ERROR] Labels file not found.")
                    return False
            else:
                self.update_status("[ERROR] Model file not found.")
                return False
        except Exception as e:
            self.update_status(f"[ERROR] Failed to load model: {str(e)}")
            return False

    def mark_attendance(self, name):
        """Mark attendance for a recognized person"""
        now = datetime.now()
        date = now.strftime('%Y-%m-%d')
        time_str = now.strftime('%H:%M:%S')
        
        # Create DataFrame for the new entry
        df = pd.DataFrame([[name, date, time_str]], columns=['Name', 'Date', 'Time'])
        
        # Check if attendance file exists
        if os.path.exists(self.attendance_file):
            # Check if this person was already marked today
            try:
                existing_df = pd.read_csv(self.attendance_file)
                today_records = existing_df[(existing_df['Name'] == name) & (existing_df['Date'] == date)]
                
                if len(today_records) == 0:
                    # Append to existing file if person not already marked today
                    df.to_csv(self.attendance_file, mode='a', header=False, index=False)
                    self.update_status(f"[INFO] Marked attendance for {name}")
                    return True
                else:
                    self.update_status(f"[INFO] {name} already marked for today")
                    return False
            except Exception as e:
                self.update_status(f"[ERROR] Failed to read attendance file: {str(e)}")
                return False
        else:
            # Create new file if it doesn't exist
            try:
                df.to_csv(self.attendance_file, index=False)
                self.update_status(f"[INFO] Created attendance file and marked {name}")
                return True
            except Exception as e:
                self.update_status(f"[ERROR] Failed to create attendance file: {str(e)}")
                return False
    
    def set_confidence_threshold(self, threshold):
        """Set the confidence threshold for face recognition"""
        try:
            threshold = float(threshold)
            if 0 <= threshold <= 100:
                self.confidence_threshold = threshold
                self.update_status(f"[INFO] Recognition threshold set to {threshold}")
                return True
            else:
                self.update_status("[ERROR] Threshold must be between 0 and 100")
                return False
        except ValueError:
            self.update_status("[ERROR] Invalid threshold value")
            return False

    def run_real_time_attendance(self):
        """Start real-time attendance monitoring in a separate thread"""
        if self.attendance_thread and self.attendance_thread.is_alive():
            self.update_status("[INFO] Attendance monitoring is already running")
            return
        
        # Reset stop flag
        self.stop_flag = False
        
        # Start attendance monitoring in a new thread
        self.attendance_thread = threading.Thread(target=self._attendance_thread)
        self.attendance_thread.daemon = True
        self.attendance_thread.start()
        
        self.update_status("[INFO] Started real-time attendance monitoring")
    
    def stop_attendance(self):
        """Stop the attendance monitoring thread"""
        if self.attendance_thread and self.attendance_thread.is_alive():
            self.stop_flag = True
            self.update_status("[INFO] Stopping attendance monitoring...")
        else:
            self.update_status("[INFO] Attendance monitoring is not running")
    
    def _attendance_thread(self):
        """Thread function for real-time attendance monitoring"""
        if not self.load_model():
            self.update_status("[ERROR] Could not load model. Please train the model first.")
            return
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            self.update_status("[ERROR] Could not open webcam")
            return
        
        # Set for tracking recognized people in current session
        recognized = set()
        
        try:
            while not self.stop_flag:
                ret, frame = cap.read()
                if not ret:
                    self.update_status("[ERROR] Failed to capture frame from webcam")
                    break
                
                # Convert to grayscale for face detection
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Apply histogram equalization for better contrast
                gray = cv2.equalizeHist(gray)
                
                # Detect faces
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))
                
                for (x, y, w, h) in faces:
                    # Extract and preprocess face
                    face = gray[y:y+h, x:x+w]
                    face = cv2.resize(face, (100, 100))
                    
                    try:
                        # Predict label for the face
                        label_id, confidence = self.recognizer.predict(face)
                        
                        # Check if prediction is confident enough
                        if confidence < self.confidence_threshold:
                            name = self.labels.get(label_id, "Unknown")
                            
                            # Mark attendance if not already recognized in this session
                            if name != "Unknown" and name not in recognized:
                                if self.mark_attendance(name):
                                    recognized.add(name)
                            
                            # Draw rectangle and label
                            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                            label_text = f"{name} ({int(confidence)})" if name != "Unknown" else "Unknown"
                            cv2.putText(frame, label_text, (x, y-10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                        else:
                            # Unknown face
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                            cv2.putText(frame, "Unknown", (x, y-10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    except Exception as e:
                        self.update_status(f"[ERROR] Recognition error: {str(e)}")
                
                # Display frame with annotations
                cv2.imshow("Attendance System", frame)
                
                # Check for key press (q to quit)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    self.stop_flag = True
                    break
                
                # Small delay to reduce CPU usage
                time.sleep(0.03)
                
        except Exception as e:
            self.update_status(f"[ERROR] Attendance monitoring error: {str(e)}")
        finally:
            cap.release()
            cv2.destroyAllWindows()
            self.update_status("[INFO] Attendance monitoring stopped")


In [28]:
def create_dataset_from_webcam(dataset_path, name, num_samples=10, delay=1.0, update_callback=None):
    """Capture face images from webcam for training dataset"""
    if not name:
        if update_callback:
            update_callback("[ERROR] Please enter a valid name")
        return False

    person_dir = os.path.join(dataset_path, name)
    os.makedirs(person_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        if update_callback:
            update_callback("[ERROR] Could not open webcam")
        return False
    
    count = 0
    if update_callback:
        update_callback(f"[INFO] Capturing images for {name}...")
    
    try:
        while count < num_samples:
            ret, frame = cap.read()
            if not ret:
                continue
            
            # Convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Apply histogram equalization
            gray = cv2.equalizeHist(gray)
            
            # Display frame with rectangle guidance
            height, width = frame.shape[:2]
            center_x, center_y = width // 2, height // 2
            offset = min(width, height) // 4
            
            # Draw positioning guide
            cv2.rectangle(frame, 
                         (center_x - offset, center_y - offset), 
                         (center_x + offset, center_y + offset), 
                         (255, 255, 0), 2)
            
            # Display instructions
            cv2.putText(frame, "Position face in box", (30, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Capturing: {count}/{num_samples}", (30, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Show frame
            cv2.imshow("Capture Face", frame)
            key = cv2.waitKey(1) & 0xFF
            
            # Press spacebar to capture, q to quit
            if key == ord(' '):
                # Detect faces
                faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))
                
                if len(faces) == 0:
                    if update_callback:
                        update_callback("[WARNING] No face detected. Please try again.")
                    continue
                
                # Take the largest face if multiple detected
                if len(faces) > 1:
                    faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)
                
                (x, y, w, h) = faces[0]
                
                # Extract and save face
                face = gray[y:y+h, x:x+w]
                face = cv2.resize(face, (100, 100))
                file_path = os.path.join(person_dir, f"{count}.jpg")
                cv2.imwrite(file_path, face)
                count += 1
                
                if update_callback:
                    update_callback(f"[INFO] Samples captured: {count}/{num_samples}")
                
                # Small delay between captures
                time.sleep(delay)
            
            elif key == ord('q'):
                break
    except Exception as e:
        if update_callback:
            update_callback(f"[ERROR] Capture error: {str(e)}")
        return False
    finally:
        cap.release()
        cv2.destroyAllWindows()
    
    if count > 0:
        if update_callback:
            update_callback(f"[INFO] Successfully captured {count} images for {name}")
        return True
    else:
        if update_callback:
            update_callback("[ERROR] No images captured")
        return False

In [29]:
def add_external_images(dataset_path, name, status_callback=None):
    """Add and process external images for a person"""
    if not name:
        if status_callback:
            status_callback("[ERROR] Please enter a valid name")
        return False

    # Open file selection dialog
    file_paths = filedialog.askopenfilenames(
        title="Select Face Images",
        filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")]
    )
    
    if not file_paths:
        if status_callback:
            status_callback("[INFO] No files selected")
        return False
    
    # Create person directory if it doesn't exist
    person_dir = os.path.join(dataset_path, name)
    os.makedirs(person_dir, exist_ok=True)
    
    # Count existing files to continue numbering
    existing_files = [f for f in os.listdir(person_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    count = len(existing_files)
    
    # Process each selected file
    success_count = 0
    for file_path in file_paths:
        try:
            # Read image
            img = cv2.imread(file_path)
            if img is None:
                if status_callback:
                    status_callback(f"[WARNING] Could not read file: {os.path.basename(file_path)}")
                continue
            
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Apply histogram equalization
            gray = cv2.equalizeHist(gray)
            
            # Detect faces
            faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))
            
            if len(faces) == 0:
                if status_callback:
                    status_callback(f"[WARNING] No face detected in {os.path.basename(file_path)}. Skipping.")
                continue
            
            # Take the largest face if multiple detected
            if len(faces) > 1:
                faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)
                if status_callback:
                    status_callback(f"[INFO] Multiple faces detected in {os.path.basename(file_path)}. Using largest face.")
            
            # Extract and save face
            (x, y, w, h) = faces[0]
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (100, 100))
            
            # Save processed face
            img_name = os.path.join(person_dir, f"{count}.jpg")
            cv2.imwrite(img_name, face_resized)
            
            if status_callback:
                status_callback(f"[INFO] Processed image: {os.path.basename(file_path)}")
            
            count += 1
            success_count += 1
            
        except Exception as e:
            if status_callback:
                status_callback(f"[ERROR] Failed to process {os.path.basename(file_path)}: {str(e)}")
    
    if status_callback:
        status_callback(f"[INFO] Successfully processed {success_count} of {len(file_paths)} images for {name}")
    
    return success_count > 0

In [30]:
def show_attendance(attendance_file):
    """Show attendance records in a new window"""
    # Create a new window
    attendance_window = tk.Toplevel()
    attendance_window.title("Attendance Records")
    attendance_window.geometry("700x500")
    
    # Create main frame
    main_frame = tk.Frame(attendance_window)
    main_frame.pack(fill="both", expand=True, padx=10, pady=10)
    
    # Create top control frame
    control_frame = tk.Frame(main_frame)
    control_frame.pack(fill="x", pady=(0, 10))
    
    # Add date filter
    date_label = tk.Label(control_frame, text="Filter by date:")
    date_label.pack(side="left", padx=(0, 5))
    
    date_var = tk.StringVar()
    date_combobox = ttk.Combobox(control_frame, textvariable=date_var, width=15)
    date_combobox.pack(side="left", padx=(0, 10))
    
    # Add name filter
    name_label = tk.Label(control_frame, text="Filter by name:")
    name_label.pack(side="left", padx=(10, 5))
    
    name_var = tk.StringVar()
    name_combobox = ttk.Combobox(control_frame, textvariable=name_var, width=15)
    name_combobox.pack(side="left")
    
    # Add search functionality
    search_label = tk.Label(control_frame, text="Search:")
    search_label.pack(side="left", padx=(10, 5))
    
    search_var = tk.StringVar()
    search_entry = tk.Entry(control_frame, textvariable=search_var, width=15)
    search_entry.pack(side="left")
    
    # Add search button
    search_button = tk.Button(control_frame, text="Search", 
                             command=lambda: apply_filters(tree, attendance_file, date_var.get(), name_var.get(), search_var.get()))
    search_button.pack(side="left", padx=5)
    
    # Add reset button
    reset_button = tk.Button(control_frame, text="Reset", 
                            command=lambda: load_data(tree, attendance_file, date_combobox, name_combobox))
    reset_button.pack(side="left")
    
    # Create a frame for the treeview and scrollbars
    tree_frame = tk.Frame(main_frame)
    tree_frame.pack(fill="both", expand=True)
    
    try:
        # Check if attendance file exists
        if not os.path.exists(attendance_file) or os.path.getsize(attendance_file) == 0:
            empty_label = tk.Label(tree_frame, text="No attendance records found", fg="gray")
            empty_label.pack(pady=50)
            return
        
        # Create Treeview
        tree = ttk.Treeview(tree_frame)
        
        # Add vertical scrollbar
        vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
        vsb.pack(side="right", fill="y")
        tree.configure(yscrollcommand=vsb.set)
        
        # Add horizontal scrollbar
        hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=tree.xview)
        hsb.pack(side="bottom", fill="x")
        tree.configure(xscrollcommand=hsb.set)
        
        # Pack tree
        tree.pack(side="left", fill="both", expand=True)
        
        # Load initial data
        load_data(tree, attendance_file, date_combobox, name_combobox)
        
        # Add export button
        export_button = tk.Button(attendance_window, text="Export to CSV", 
                                                           command=lambda: export_data(tree))
        export_button.pack(pady=10)
        
    except Exception as e:
        error_label = tk.Label(tree_frame, text=f"Error loading attendance data: {str(e)}", fg="red")
        error_label.pack(pady=20)

def load_data(tree, attendance_file, date_combobox=None, name_combobox=None):
    """Load attendance data into the treeview"""
    # Clear existing data
    for item in tree.get_children():
        tree.delete(item)
    
    # Clear and configure columns
    tree["columns"] = []
    tree["show"] = "headings"
    
    try:
        # Load the attendance data
        df = pd.read_csv(attendance_file)
        
        # Configure columns based on DataFrame
        columns = list(df.columns)
        tree["columns"] = columns
        
        # Set column headings and widths
        for col in columns:
            tree.heading(col, text=col, anchor="w")
            # Determine column width based on content
            col_width = max(len(str(col)) * 10, df[col].astype(str).str.len().max() * 10)
            tree.column(col, width=min(col_width, 200), anchor="w")
        
        # Insert data
        for idx, row in df.iterrows():
            values = row.tolist()
            tree.insert("", "end", values=values)
        
        # Update date and name filter options if comboboxes are provided
        if date_combobox is not None and 'Date' in df.columns:
            dates = sorted(df['Date'].unique().tolist())
            date_combobox['values'] = [''] + dates
        
        if name_combobox is not None and 'Name' in df.columns:
            names = sorted(df['Name'].unique().tolist())
            name_combobox['values'] = [''] + names
            
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load attendance data: {str(e)}")

def apply_filters(tree, attendance_file, date_filter=None, name_filter=None, search_text=None):
    """Apply filters to the attendance data"""
    # Clear existing data
    for item in tree.get_children():
        tree.delete(item)
    
    try:
        # Load the attendance data
        df = pd.read_csv(attendance_file)
        
        # Apply date filter if specified
        if date_filter and date_filter != '':
            df = df[df['Date'] == date_filter]
        
        # Apply name filter if specified
        if name_filter and name_filter != '':
            df = df[df['Name'] == name_filter]
        
        # Apply search filter if specified
        if search_text and search_text != '':
            # Search in all columns
            mask = pd.DataFrame(False, index=df.index, columns=['match'])
            for col in df.columns:
                mask['match'] |= df[col].astype(str).str.contains(search_text, case=False, na=False)
            df = df[mask['match']]
        
        # Insert filtered data
        for idx, row in df.iterrows():
            values = row.tolist()
            tree.insert("", "end", values=values)
            
    except Exception as e:
        messagebox.showerror("Error", f"Failed to apply filters: {str(e)}")

def export_data(tree):
    """Export currently displayed data to CSV"""
    file_path = filedialog.asksaveasfilename(
        defaultextension='.csv',
        filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
        title="Export Attendance Data"
    )
    
    if not file_path:
        return
    
    try:
        # Get column headers
        columns = tree["columns"]
        
        # Get all rows
        data = []
        for item in tree.get_children():
            values = tree.item(item)['values']
            data.append(values)
        
        # Create DataFrame and save to CSV
        df = pd.DataFrame(data, columns=columns)
        df.to_csv(file_path, index=False)
        
        messagebox.showinfo("Export Successful", f"Data exported to {file_path}")
        
    except Exception as e:
        messagebox.showerror("Export Error", f"Failed to export data: {str(e)}")

In [31]:
def start_gui(system):
    """Start the main GUI for the face attendance system"""
    # Create the main window
    root = Tk()
    root.title("Advanced Face Attendance System")
    root.geometry("800x600")
    
    # Set icon (if available)
    try:
        root.iconbitmap("icon.ico")  # Replace with your icon path
    except:
        pass  # Icon file not available
    
    # Create tabs
    tab_control = ttk.Notebook(root)
    
    # Tab 1: Dataset Management
    tab_dataset = ttk.Frame(tab_control)
    tab_control.add(tab_dataset, text="Dataset Management")
    
    # Tab 2: Training & Recognition
    tab_recognition = ttk.Frame(tab_control)
    tab_control.add(tab_recognition, text="Training & Recognition")
    
    # Tab 3: Settings
    tab_settings = ttk.Frame(tab_control)
    tab_control.add(tab_settings, text="Settings")
    
    tab_control.pack(expand=1, fill="both")
    
    # Status bar
    status_frame = Frame(root, relief=tk.SUNKEN, bd=1)
    status_frame.pack(side=tk.BOTTOM, fill=tk.X)
    
    status_label = Label(status_frame, text="Ready", anchor=tk.W, padx=5)
    status_label.pack(side=tk.LEFT, fill=tk.X)
    
    # Function to update status
    def update_status(message):
        status_label.config(text=message)
        root.update_idletasks()
    
    # Set status callback
    system.set_status_callback(update_status)
    
    # ===== TAB 1: DATASET MANAGEMENT =====
    # Name entry
    Label(tab_dataset, text="Person Name:").grid(row=0, column=0, padx=10, pady=10, sticky="w")
    name_entry = Entry(tab_dataset, width=30)
    name_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")
    
    # Number of samples
    Label(tab_dataset, text="Number of Samples:").grid(row=1, column=0, padx=10, pady=10, sticky="w")
    samples_var = tk.StringVar(value="10")
    samples_entry = Entry(tab_dataset, textvariable=samples_var, width=10)
    samples_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")
    
    # Capture faces button
    def capture_webcam():
        try:
            num_samples = int(samples_var.get())
            if num_samples <= 0:
                update_status("[ERROR] Number of samples must be positive")
                return
            create_dataset_from_webcam(DATASET_PATH, name_entry.get().strip(), num_samples, 1.0, update_status)
        except ValueError:
            update_status("[ERROR] Invalid number of samples")

    Button(tab_dataset, text="Capture via Webcam", width=20,
           command=capture_webcam).grid(row=2, column=0, padx=10, pady=10)
    
    # Add external images button
    Button(tab_dataset, text="Add External Images", width=20,
           command=lambda: add_external_images(DATASET_PATH, name_entry.get().strip(), update_status)).grid(row=2, column=1, padx=10, pady=10)
    
    # View dataset button
    Button(tab_dataset, text="View Dataset", width=20,
           command=lambda: os.startfile(DATASET_PATH) if os.name == 'nt' else 
                          os.system(f'xdg-open "{DATASET_PATH}"')).grid(row=3, column=0, padx=10, pady=10)
    
    # Delete person button
    def delete_person():
        """Delete a person's dataset"""
        name = name_entry.get().strip()
        if not name:
            update_status("[ERROR] Please enter a name")
            return
        
        person_dir = os.path.join(DATASET_PATH, name)
        if os.path.exists(person_dir):
            try:
                shutil.rmtree(person_dir)
                update_status(f"[INFO] Successfully deleted dataset for {name}")
            except Exception as e:
                update_status(f"[ERROR] Failed to delete dataset: {str(e)}")
        else:
            update_status(f"[ERROR] No dataset found for {name}")

    Button(tab_dataset, text="Delete Person", width=20,
           command=delete_person).grid(row=3, column=1, padx=10, pady=10)

    # ===== TAB 2: TRAINING & RECOGNITION =====
    # Train model button
    Button(tab_recognition, text="Train Model", width=20,
           command=system.train_model).grid(row=0, column=0, padx=10, pady=10)

    # Start attendance button
    Button(tab_recognition, text="Start Attendance", width=20,
           command=system.run_real_time_attendance).grid(row=1, column=0, padx=10, pady=10)

    # Stop attendance button
    Button(tab_recognition, text="Stop Attendance", width=20,
           command=system.stop_attendance).grid(row=1, column=1, padx=10, pady=10)

    # View attendance button
    Button(tab_recognition, text="View Attendance", width=20,
           command=lambda: show_attendance(ATTENDANCE_FILE)).grid(row=2, column=0, padx=10, pady=10)

    # View log button
    Button(tab_recognition, text="View Log", width=20,
           command=lambda: os.startfile(LOG_FILE) if os.name == 'nt' else 
                          os.system(f'xdg-open "{LOG_FILE}"')).grid(row=2, column=1, padx=10, pady=10)

    # ===== TAB 3: SETTINGS =====
    # Confidence threshold
    Label(tab_settings, text="Confidence Threshold (0-100):").grid(row=0, column=0, padx=10, pady=10, sticky="w")
    threshold_var = tk.StringVar(value=str(system.confidence_threshold))
    threshold_entry = Entry(tab_settings, textvariable=threshold_var, width=10)
    threshold_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")

    # Set threshold button
    Button(tab_settings, text="Set Threshold", width=20,
           command=lambda: system.set_confidence_threshold(threshold_var.get())).grid(row=1, column=0, padx=10, pady=10)

    # Clear attendance button
    def clear_attendance():
        """Clear all attendance records"""
        if messagebox.askyesno("Confirm", "Are you sure you want to clear all attendance records?"):
            try:
                if os.path.exists(ATTENDANCE_FILE):
                    os.remove(ATTENDANCE_FILE)
                    update_status("[INFO] Attendance records cleared")
                else:
                    update_status("[INFO] No attendance records to clear")
            except Exception as e:
                update_status(f"[ERROR] Failed to clear attendance: {str(e)}")

    Button(tab_settings, text="Clear Attendance", width=20,
           command=clear_attendance).grid(row=2, column=0, padx=10, pady=10)

    # Clear log button
    def clear_log():
        """Clear the system log"""
        if messagebox.askyesno("Confirm", "Are you sure you want to clear the system log?"):
            try:
                if os.path.exists(LOG_FILE):
                    open(LOG_FILE, 'w').close()
                    update_status("[INFO] System log cleared")
                else:
                    update_status("[INFO] No log file to clear")
            except Exception as e:
                update_status(f"[ERROR] Failed to clear log: {str(e)}")

    Button(tab_settings, text="Clear Log", width=20,
           command=clear_log).grid(row=2, column=1, padx=10, pady=10)

    # Make window responsive
    for i in range(4):
        tab_dataset.grid_rowconfigure(i, weight=1)
        tab_recognition.grid_rowconfigure(i, weight=1)
        tab_settings.grid_rowconfigure(i, weight=1)
    for i in range(2):
        tab_dataset.grid_columnconfigure(i, weight=1)
        tab_recognition.grid_columnconfigure(i, weight=1)
        tab_settings.grid_columnconfigure(i, weight=1)

    # Start the main loop
    root.mainloop()

In [32]:
if __name__ == "__main__":
    try:
        # Initialize the attendance system
        attendance_system = CSVAttendanceSystem()
        
        # Start the GUI
        start_gui(attendance_system)
    except Exception as e:
        print(f"Failed to start application: {str(e)}")
        messagebox.showerror("Startup Error", f"Failed to start application: {str(e)}")

[2025-04-20 15:27:40] [INFO] Training model...
[2025-04-20 15:27:40] [INFO] Collected 10 images for Chetan
[2025-04-20 15:27:40] [INFO] Model trained successfully with 10 images.
[2025-04-20 15:27:45] [INFO] Started real-time attendance monitoring
[2025-04-20 15:27:45] [INFO] Model and labels loaded successfully.
[2025-04-20 15:27:46] [INFO] Created attendance file and marked Chetan
[2025-04-20 15:28:26] [INFO] Attendance monitoring stopped
[2025-04-20 15:28:49] [INFO] Recognition threshold set to 50.0
[2025-04-20 15:29:21] [INFO] Training model...
[2025-04-20 15:29:21] [INFO] Collected 10 images for Chetan
[2025-04-20 15:29:21] [INFO] Collected 3 images for Chirag
[2025-04-20 15:29:21] [INFO] Model trained successfully with 13 images.
[2025-04-20 15:29:24] [INFO] Started real-time attendance monitoring
[2025-04-20 15:29:24] [INFO] Model and labels loaded successfully.
[2025-04-20 15:29:51] [INFO] Attendance monitoring stopped
[2025-04-20 15:29:57] [INFO] Training model...
[2025-04-20 